# ch10 Debugging

In [1]:
raise Exception('This is the error message.')

Exception: This is the error message.

In [2]:
def boxPrint(symbol, width, height):
    if len(symbol) != 1:
        raise Exception('Symbol must be a single chracater string.')
    if width <= 2:
        raise Exception('Width must be greater than 2.')
    if height <= 2:
        raise Exception('Height must be greater than 2.')
    print(symbol * width)
    for i in range(height - 2):
        print(symbol + (' ' * (width - 2)) + symbol)
    print(symbol * width)
    
for sym, w, h in (('*', 4, 4), ('O', 20, 5), ('x', 1, 3), ('ZZ', 3, 3)):
    try:
        boxPrint(sym, w, h)
    except Exception as err:
        print('An exception happened: ' + str(err))

****
*  *
*  *
****
OOOOOOOOOOOOOOOOOOOO
O                  O
O                  O
O                  O
OOOOOOOOOOOOOOOOOOOO
An exception happened: Width must be greater than 2.
An exception happened: Symbol must be a single chracater string.


- boxPrint 라고 함수명에서 이야기 해줬잖아.
- 그런데 왜 결과물을 보고 해석하려고 하냐? 함수명으로 유추했으면 좀 더 빨리 이해가 됐을거 아니야?

## Getting the Traceback as a String

In [3]:
def spam():
    bacon()
    
def bacon():
    raise Exception('This is the error message.')
    
spam()

Exception: This is the error message.

In [4]:
import traceback

In [5]:
try:
    raise Exception('This is the error message.')
except:
    errorFile = open('errorInfo.txt', 'w')
    errorFile.write(traceback.format_exc())
    errorFile.close()
    print('The traceback info was written to errorInfo.txt.')

The traceback info was written to errorInfo.txt.


In [6]:
!cat errorInfo.txt

Traceback (most recent call last):
  File "<ipython-input-5-9761f8ef0b4a>", line 2, in <module>
    raise Exception('This is the error message.')
Exception: This is the error message.


## Assertions

- A condition(True or False를 평가한다)
- A comma
- condition is False 일 때 화면에 보인다.

In [7]:
podBayDoorStatus = 'open'

In [8]:
assert podBayDoorStatus == 'open', 'The pod bay doors need to be "open".'

In [9]:
podBayDoorStatus = 'I\'m sorry, Dave. I\'m afraid I can\' do that.'

In [10]:
assert podBayDoorStatus == 'open', 'The pod bay doors need to be "open".'

AssertionError: The pod bay doors need to be "open".

In [11]:
market_2nd = {'ns': 'green', 'ew': 'red'}
mission_16th = {'ns': 'red', 'ew': 'green'}

In [28]:
def switchLights(stoplight):
    for key in stoplight.keys():
        if stoplight[key] == 'green':
            stoplight[key] = 'yellow'
        elif stoplight[key] == 'yellow':
            stoplight[key] = 'red'
        elif stoplight[key] == 'red':
            stoplight[key] = 'green'
            
    assert 'red' in stoplight.values(), 'Neither light is red! ' + str(stoplight)
            
switchLights(market_2nd)

AssertionError: Neither light is red! {'ew': 'green', 'ns': 'yellow'}

- AssertionError가 빨리나면 좋다. 그만큼 디버깅하는 노력을 줄일 수 있기 때문에.

### Disabling Assertions

- 파이썬을 실행할 때 -O option을 주면 Assertion을 disable 할 수 있다.
- 다른 사람이 프로그램을 실행할 때를 봐야한다. -O option을 사용해서 이상하지 않은 프로그램을 어떻게 실행하는지 Appendix B를 봐라

## Logging

- print() 는 너의 코드를 디버깅 할 수 있다.
- Logging은 너의 프로그램에서 무슨 일이 발생했는지 알기 위한 좋은 방법

### Using the logging Module



In [29]:
import logging
logging.basicConfig(level=logging.DEBUG, format=' %(asctime)s - %(levelname)s - %(message)s')

In [40]:
%%writefile test_logging.py
import logging
logging.basicConfig(level=logging.DEBUG, format=' %(asctime)s - %(levelname)s - %(message)s')
logging.debug('Start of program')

def factorial(n):
    logging.debug('Start of factorial( %s)' % (n))
    total = 1
    for i in range(1, n + 1):
        total *= i
        logging.debug('i is ' + str(i) + ', total is ' + str(total))
    logging.debug('End of factorial( %s)' % (n))
    return total

print(factorial(5))
logging.debug('End of program')

Overwriting test_logging.py


In [41]:
!python test_logging.py

 2015-06-02 17:22:33,412 - DEBUG - Start of program
 2015-06-02 17:22:33,412 - DEBUG - Start of factorial( 5)
 2015-06-02 17:22:33,412 - DEBUG - i is 1, total is 1
 2015-06-02 17:22:33,412 - DEBUG - i is 2, total is 2
 2015-06-02 17:22:33,412 - DEBUG - i is 3, total is 6
 2015-06-02 17:22:33,412 - DEBUG - i is 4, total is 24
 2015-06-02 17:22:33,412 - DEBUG - i is 5, total is 120
 2015-06-02 17:22:33,412 - DEBUG - End of factorial( 5)
120
 2015-06-02 17:22:33,412 - DEBUG - End of program


### Don't Debug with print()

- 보고 싶지 않을 떄는 logging.disable(logging.CRITICAL) 호출하면 됨

### Logging Levels

#### Table 10-1. Logging Levels in Python

Level | Logging Function | Description
--- | --- | ---
DEBUG | logging.debug() | The lowest level. Used for small details. Usaually you care about these messages only when diagnosing problems.
INFO | logging.info() | Used to record information on general events in your program or confirm that things are working at their point in the program.
WARNING | logging.warning() | Used to indicate a potential problem that doesn't prevent the program from working but might do so in the future.
ERROR | logging.error() | Used to record an error that caused the program to fail to do something.
CRITICAl | logging.critical() | The highest level. Used to indicate a fatal error that ahs caused or is about to cause the program to stop running entirely.

In [45]:
%%writefile test_logging_debug.py
import logging
logging.basicConfig(level=logging.DEBUG, format=' %(asctime)s - %(levelname)s - %(message)s')

logging.debug('Some debugging details.')
logging.info('The logging module is working.')
logging.warning('An error message is about to be logged.')
logging.error('An error has occurred.')
logging.critical('The program is unable to recover!')

Writing test_logging_debug.py


In [47]:
!python test_logging_debug.py

 2015-06-02 17:33:04,396 - DEBUG - Some debugging details.
 2015-06-02 17:33:04,396 - INFO - The logging module is working.
 2015-06-02 17:33:04,396 - WARNING - An error message is about to be logged.
 2015-06-02 17:33:04,396 - ERROR - An error has occurred.
 2015-06-02 17:33:04,396 - CRITICAL - The program is unable to recover!


In [49]:
%%writefile test_logging_info.py
import logging
logging.basicConfig(level=logging.INFO, format=' %(asctime)s - %(levelname)s - %(message)s')

logging.debug('Some debugging details.')
logging.info('The logging module is working.')
logging.warning('An error message is about to be logged.')
logging.error('An error has occurred.')
logging.critical('The program is unable to recover!')

Overwriting test_logging_info.py


In [50]:
!python test_logging_info.py

 2015-06-02 17:33:24,577 - INFO - The logging module is working.
 2015-06-02 17:33:24,577 - WARNING - An error message is about to be logged.
 2015-06-02 17:33:24,577 - ERROR - An error has occurred.
 2015-06-02 17:33:24,578 - CRITICAL - The program is unable to recover!


In [51]:
%%writefile test_logging_error.py
import logging
logging.basicConfig(level=logging.ERROR, format=' %(asctime)s - %(levelname)s - %(message)s')

logging.debug('Some debugging details.')
logging.info('The logging module is working.')
logging.warning('An error message is about to be logged.')
logging.error('An error has occurred.')
logging.critical('The program is unable to recover!')

Writing test_logging_error.py


In [52]:
!python test_logging_error.py

 2015-06-02 17:33:47,562 - ERROR - An error has occurred.
 2015-06-02 17:33:47,562 - CRITICAL - The program is unable to recover!


### Disabling Logging



In [58]:
%%writefile test_logging_disable.py
import logging
logging.basicConfig(level=logging.INFO, format=' %(asctime)s - %(levelname)s - %(message)s')

logging.critical('Critical error! Critical error!')
# logging.disable(logging.CRITICAL)
logging.critical('Critical error! Critical error!')
logging.error('Error! Error!')

Overwriting test_logging_disable.py


In [59]:
!python test_logging_disable.py

 2015-06-02 17:38:54,868 - CRITICAL - Critical error! Critical error!
 2015-06-02 17:38:54,868 - CRITICAL - Critical error! Critical error!
 2015-06-02 17:38:54,868 - ERROR - Error! Error!


In [60]:
%%writefile test_logging_disable2.py
import logging
logging.basicConfig(level=logging.INFO, format=' %(asctime)s - %(levelname)s - %(message)s')

logging.critical('Critical error! Critical error!')
logging.disable(logging.CRITICAL)
logging.critical('Critical error! Critical error!')
logging.error('Error! Error!')

Writing test_logging_disable2.py


In [61]:
!python test_logging_disable2.py

 2015-06-02 17:39:23,812 - CRITICAL - Critical error! Critical error!


In [62]:
%%writefile test_logging_disable3.py
import logging
logging.basicConfig(level=logging.INFO, format=' %(asctime)s - %(levelname)s - %(message)s')

logging.critical('Critical error! Critical error!')
logging.disable(logging.INFO)
logging.critical('Critical error! Critical error!')
logging.error('Error! Error!')

Writing test_logging_disable3.py


In [63]:
!python test_logging_disable3.py

 2015-06-02 17:39:40,643 - CRITICAL - Critical error! Critical error!
 2015-06-02 17:39:40,643 - CRITICAL - Critical error! Critical error!
 2015-06-02 17:39:40,643 - ERROR - Error! Error!


In [64]:
%%writefile test_logging_disable4.py
import logging
logging.basicConfig(level=logging.INFO, format=' %(asctime)s - %(levelname)s - %(message)s')

logging.critical('Critical error! Critical error!')
logging.disable(logging.ERROR)
logging.critical('Critical error! Critical error!')
logging.error('Error! Error!')

Writing test_logging_disable4.py


In [65]:
!python test_logging_disable4.py

 2015-06-02 17:40:02,170 - CRITICAL - Critical error! Critical error!
 2015-06-02 17:40:02,170 - CRITICAL - Critical error! Critical error!


#### disable 범위

```python
logging.disable(logging.ERROR)
```

- 라고 해주면 ERROR 밑으로는 모두 disable 되네. 가장 위험한 CRITICAL만 보임

### Logging to a File



In [111]:
%%writefile test_logging_file.py
import logging
logging.basicConfig(filename='myProgramLog.txt', level=logging.INFO, format=' %(asctime)s - %(levelname)s - %(message)s')
logging.critical('Critical error! Critical error!')
logging.critical('Critical error! Critical error!')
logging.error('Error! Error!')
logging.debug('Debug!')
logging.info('Info!')

Overwriting test_logging_file.py


In [112]:
!rm -rf myProgramLog.txt

In [113]:
!python test_logging_file.py

In [114]:
!ls -l myProgramLog.txt

-rw-r--r--  1 re4lfl0w  staff  229  6  2 17:51 myProgramLog.txt


In [115]:
!cat myProgramLog.txt

 2015-06-02 17:51:13,041 - CRITICAL - Critical error! Critical error!
 2015-06-02 17:51:13,041 - CRITICAL - Critical error! Critical error!
 2015-06-02 17:51:13,041 - ERROR - Error! Error!
 2015-06-02 17:51:13,041 - INFO - Info!


In [143]:
import random
heads = 0

# 아 1000번을 돈 다음에 heads를 찍어주니 500 전후겠네
for i in range(1, 1001):
    if random.randint(0, 1) == 1:
        heads += 1
    if i == 500:
        print('Halfway done!')
print('Heads came up ' + str(heads) + ' times.')

Halfway done!
Heads came up 533 times.


In [160]:
cnt = 0
for i in range(10):
    ran = random.randint(0, 1)
    if ran:
        print '1'
        cnt += 1
print ''
print cnt

1
1
1
1
1
1

6


## Summary

- Assertions, exceptions, logging, debugger는 모두 버그를 찾고 막아줄 수 있는 유용한 툴이다.
- try, except를 써서 예외를 제어할 수 있다. 
- 로깅 모듈은 너의 코드에서 무슨 일이 발생한지 확인할 수 있는 좋은 방법이다.
- print() 보다 좋다. 왜냐하면 각각 다른 logging level을 쓸 수 있고 text file로 저장할 수 있기 때문

## Practice Questions

    1. Write an assert statement that triggers an AssertionError if the variable spam is an integer less than 10

In [174]:
spam = 15

In [175]:
assert spam < 10

AssertionError: 

    2. Write an assert statement that triggers an AssertionError if the variables eggs and bacon contain strings that are the same as each other, even if their cases are different(that is, 'hello' and 'hello' are considered the same, and 'goodbye' and 'GOODbye' are also considered the same).

In [172]:
eggs = 'eggs'
bacon = 'Eggs'

In [173]:
assert eggs.lower() == bacon.lower()

    3. Write an assert statement that always triggers an AssertionError.

### Debuggin Con Toss

In [198]:
import random
guess = ''

while guess not in ('heads', 'tails'):
    print('Guess the coin toss! Enter heads or tails:')
    guess = raw_input()
toss = random.randint(0, 1)
if toss == guess:
    print('You got it!')
else:
    print('Nope! Guess again!')
    guess = raw_input()
    if toss == guess:
        print('You got it!')
    else:
        print('Nope. You are really bad at this game.')

heads
tails


#### 동전 맞추기 Logic

1. 사용자에게 앞면인지 뒷면인지 맞춰보라고 알린다.
2. 사용자의 입력을 받는다.
3. 사용자의 입력과 동전의 면이 일치하는지 확인한다.
4. 동일하다면 축하하고 2번 실패하면 미안하다고 종료한다.

In [16]:
%%writefile test_debug.py
import random
guess = ''

while guess not in ('heads', 'tails'):
    print('Guess the coin toss! Enter heads or tails:')
    guess = raw_input()
toss = random.randint(0, 1)

def return_guess(guess):
    if guess == 'heads':
        guess = 1
    else:
        guess = 0
    return guess
guess = return_guess(guess)
    
if toss == guess:
    print('You got it!')
else:
    print('Nope! Guess again!')
    guess = raw_input()
    if toss == return_guess(guess):
        print('You got it!')
    else:
        print('Nope. You are really bad at this game.')

Overwriting test_debug.py


In [22]:
%run test_debug.py

Guess the coin toss! Enter heads or tails:
tails
Nope! Guess again!
heads
You got it!
